In [29]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [30]:
# Dictionaries for factories (supply in tons), depots(throughput in tons), customers(demand in tons)
f = dict({
    'Liverpool': 150000,
    'Brighton': 200000
})

d = dict({
    'Newcastle': 70000,
    'Birmingham': 50000,
    'London': 100000,
    'Exeter': 40000
})

c = dict({
    'C1': 50000,
    'C2': 10000,
    'C3': 40000,
    'C4': 35000,
    'C5': 60000,
    'C6': 20000
})

In [31]:
# Dictionary for distribution costs per factory (in €/ton)

# all unprefered arcs are deleted, except Exeter-C5 and London-C5

arcs, distr_cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C4'): 1.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5
})

In [32]:
# Dictionary for distribution costs per factory (in €/ton)

# all unprefered arcs are deleted, except London-C5

# arcs, distr_cost = gp.multidict({
#     ('Liverpool', 'Newcastle'): 0.5,
#     ('Liverpool', 'Birmingham'): 0.5,
#     ('Liverpool', 'London'): 1.0,
#     ('Liverpool', 'Exeter'): 0.2,
#     ('Liverpool', 'C1'): 1.0,
#     ('Liverpool', 'C3'): 1.5,
#     ('Liverpool', 'C4'): 2.0,
#     ('Brighton', 'Birmingham'): 0.3,
#     ('Brighton', 'London'): 0.5,
#     ('Brighton', 'Exeter'): 0.2,
#     ('Newcastle', 'C2'): 1.5,
#     ('Newcastle', 'C3'): 0.5,
#     ('Newcastle', 'C4'): 1.5,
#     ('Birmingham', 'C3'): 0.5,
#     ('Birmingham', 'C4'): 1.0,
#     ('Birmingham', 'C5'): 0.5,
#     ('London', 'C3'): 2.0,
#     ('London', 'C5'): 0.5,
#     ('London', 'C6'): 1.5,
#     ('Exeter', 'C3'): 0.2,
#     ('Exeter', 'C4'): 1.5,
#     ('Exeter', 'C6'): 1.5
# })

In [33]:
# Dictionary for distribution costs per factory (in €/ton)

# all unprefered arcs are deleted, except Exeter-C5

# arcs, distr_cost = gp.multidict({
#     ('Liverpool', 'Newcastle'): 0.5,
#     ('Liverpool', 'Birmingham'): 0.5,
#     ('Liverpool', 'London'): 1.0,
#     ('Liverpool', 'Exeter'): 0.2,
#     ('Liverpool', 'C1'): 1.0,
#     ('Liverpool', 'C3'): 1.5,
#     ('Liverpool', 'C4'): 2.0,
#     ('Brighton', 'Birmingham'): 0.3,
#     ('Brighton', 'London'): 0.5,
#     ('Brighton', 'Exeter'): 0.2,
#     ('Newcastle', 'C2'): 1.5,
#     ('Newcastle', 'C3'): 0.5,
#     ('Newcastle', 'C4'): 1.5,
#     ('Birmingham', 'C3'): 0.5,
#     ('Birmingham', 'C4'): 1.0,
#     ('Birmingham', 'C5'): 0.5,
#     ('London', 'C3'): 2.0,
#     ('London', 'C6'): 1.5,
#     ('Exeter', 'C3'): 0.2,
#     ('Exeter', 'C4'): 1.5,
#     ('Exeter', 'C5'): 0.5,
#     ('Exeter', 'C6'): 1.5
# })

In [34]:
# Define model
model = Model('Transshipment_Assignment')

# Create decision variables
plan = model.addVars(arcs, name="plan")

# set objective function
model.setObjective(plan.prod(distr_cost), GRB.MINIMIZE)

In [35]:
# Define constraints
# maximal factory supply
factories = f.keys()
factory_con = model.addConstrs((gp.quicksum(plan.select(i, '*')) <= f[i] for i in factories), name="factory_constraint")

# maximal depot throughput
depots = d.keys()
depot_through_con = model.addConstrs((gp.quicksum(plan.select(j, '*')) <= d[j] for j in depots), 
                                     name="depot_through_constraint")
# depot output == depot input
depot_out_con = model.addConstrs((gp.quicksum(plan.select('*', j)) == gp.quicksum(plan.select(j, '*')) for j in depots),
                                                                                 name="depot_out_constraint")
# customer requirement == customer input
customers = c.keys()
customer_con = model.addConstrs((gp.quicksum(plan.select('*', k)) == c[k] for k in customers), name="customer_constraint")

In [36]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 23 columns and 59 nonzeros
Model fingerprint: 0x596d4055
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 4 rows and 4 columns
Presolve time: 0.03s
Presolved: 12 rows, 19 columns, 50 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7300000e+05   3.374375e+04   0.000000e+00      0s
       5    2.2850000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.05 seconds
Optimal objective  2.285000000e+05


In [37]:
# Optimal total distribution costs per month (in €)
obj = model.getObjective()
print("Total distribution costs per month: {0} €".format(obj.getValue()))

Total distribution costs per month: 228500.0 €


In [38]:
# Optimal monthly transportation plan in tons
# Extract quantities into a separate list
quantities=[]
for i in arcs:
    quantities.append(plan[i].x)
quantities

# Extract arcs into a separate list
arcs_list=list(arcs)

# Define a dataframe for arcs
df = pd.DataFrame(arcs_list, columns=["Supplier", "Receiver"])

# Add column for quantities
df["Quantity (in tons)"]=quantities

# Styling of dataframe
zero_columns=df[df["Quantity (in tons)"] == 0].index
df.drop(zero_columns, inplace=True)
trans_plan = df.round(2)
trans_plan

,Supplier,Receiver,Quantity (in tons)
0,Liverpool,Newcastle,10000.0
3,Liverpool,Exeter,40000.0
4,Liverpool,C1,50000.0
7,Brighton,Birmingham,50000.0
8,Brighton,London,65000.0
10,Newcastle,C2,10000.0
14,Birmingham,C4,35000.0
15,Birmingham,C5,15000.0
17,London,C5,45000.0
18,London,C6,20000.0
